In [59]:
import torch
import os
from PIL import Image
from monai.transforms import (
    Compose,
    LoadImage,
    RandRotate90,
    RandSpatialCrop,
    ScaleIntensity
)
from torchvision import transforms
from monai.data import ArrayDataset, create_test_image_2d, decollate_batch, DataLoader
from torchinfo import summary

In [7]:
os.listdir('.')

['Ex8_colab(2).ipynb', '.gitkeep', 'villi', 'load_model_test.ipynb']

In [43]:
unet_model = torch.jit.load('../models/model.pt').to('cpu')

In [9]:
unet_model.eval()

RecursiveScriptModule(
  original_name=UNETModel
  (net): RecursiveScriptModule(
    original_name=UNet
    (model): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=ResidualUnit
        (conv): RecursiveScriptModule(
          original_name=Sequential
          (unit0): RecursiveScriptModule(
            original_name=Convolution
            (conv): RecursiveScriptModule(original_name=Conv2d)
            (adn): RecursiveScriptModule(
              original_name=ADN
              (N): RecursiveScriptModule(original_name=InstanceNorm2d)
              (D): RecursiveScriptModule(original_name=Dropout)
              (A): RecursiveScriptModule(original_name=PReLU)
            )
          )
          (unit1): RecursiveScriptModule(
            original_name=Convolution
            (conv): RecursiveScriptModule(original_name=Conv2d)
            (adn): RecursiveScriptModule(
              original_name=ADN
              (N): Recursiv

In [60]:
summary(unet_model, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                                                                          Output Shape              Param #
UNETModel                                                                                       --                        --
├─UNet: 1-1                                                                                     --                        --
│    └─Sequential: 2-1                                                                          --                        --
│    │    └─ResidualUnit: 3-1                                                                   --                        3,218
│    │    └─SkipConnection: 3-2                                                                 --                        1,621,902
│    │    └─Sequential: 3-3                                                                     --                        952
├─DiceCELoss: 1-2                                                                               --           

In [10]:
type(unet_model)

torch.jit._script.RecursiveScriptModule

In [12]:
img = Image.open('../data/raw/BCSS/test/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0_0_0_size224.png')

In [36]:
# img_transf = transforms.ToTensor()
img_transf = Compose(
    [
        transforms.Resize((224, 224)),  # Adjust size according to your model's input size
        transforms.ToTensor()
        #ScaleIntensity()
    ]
)

In [39]:
img_transf(img)

tensor([[[0.6980, 0.6941, 0.6863,  ..., 0.8510, 0.8549, 0.8471],
         [0.6863, 0.6863, 0.6941,  ..., 0.8275, 0.8431, 0.8431],
         [0.7137, 0.6941, 0.6941,  ..., 0.8275, 0.8353, 0.8392],
         ...,
         [0.3922, 0.3922, 0.4118,  ..., 0.9333, 0.9216, 0.8784],
         [0.3804, 0.3529, 0.3804,  ..., 0.8706, 0.8510, 0.8039],
         [0.3922, 0.3451, 0.3647,  ..., 0.8157, 0.7843, 0.7451]],

        [[0.4824, 0.4980, 0.5255,  ..., 0.7059, 0.7098, 0.6824],
         [0.4745, 0.4941, 0.5176,  ..., 0.7098, 0.7255, 0.7216],
         [0.4588, 0.4510, 0.4510,  ..., 0.6902, 0.7059, 0.7059],
         ...,
         [0.2745, 0.2275, 0.2078,  ..., 0.7490, 0.7255, 0.6784],
         [0.2745, 0.2431, 0.2353,  ..., 0.6745, 0.6392, 0.5961],
         [0.2431, 0.2235, 0.2353,  ..., 0.5804, 0.5529, 0.5333]],

        [[0.7059, 0.6902, 0.6980,  ..., 0.8275, 0.8392, 0.8392],
         [0.7020, 0.7098, 0.7412,  ..., 0.8118, 0.8196, 0.8431],
         [0.7098, 0.7059, 0.7373,  ..., 0.8157, 0.8196, 0.

In [44]:
unet_model(img_transf(img))

code/__torch__/torch/nn/modules/instancenorm.py:23: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
code/__torch__/torch/nn/modules/instancenorm/___torch_mangle_3.py:23: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
code/__torch__/torch/nn/modules/instancenorm/___torch_mangle_11.py:23: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
code/__torch__/torch/nn/modules/instancenorm/___torch_mangle_19.py:23: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
code/__torch__/torch/nn/modules/instancenorm/___torch_mangle_27.py:23: UserWarni

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/breast_cancer_segmentation/models/UNETModel.py", line 23, in forward
    input: Tensor) -> Tensor:
    net = self.net
    return (net).forward(input, )
            ~~~~~~~~~~~~ <--- HERE
  File "code/__torch__/monai/networks/nets/unet.py", line 21, in forward
    x: Tensor) -> Tensor:
    model = self.model
    return (model).forward(x, )
            ~~~~~~~~~~~~~~ <--- HERE
  File "code/__torch__/torch/nn/modules/container/___torch_mangle_64.py", line 15, in forward
    _2 = getattr(self, "2")
    input0 = (_0).forward(input, )
    input1 = (_1).forward(input0, )
              ~~~~~~~~~~~ <--- HERE
    return (_2).forward(input1, )
  def __len__(self: __torch__.torch.nn.modules.container.___torch_mangle_64.Sequential) -> int:
  File "code/__torch__/monai/networks/layers/simplelayers/___torch_mangle_54.py", line 13, in fo

In [42]:
unet_model.

In [56]:
img_transf(img).squeeze(0)

tensor([[[0.6980, 0.6941, 0.6863,  ..., 0.8510, 0.8549, 0.8471],
         [0.6863, 0.6863, 0.6941,  ..., 0.8275, 0.8431, 0.8431],
         [0.7137, 0.6941, 0.6941,  ..., 0.8275, 0.8353, 0.8392],
         ...,
         [0.3922, 0.3922, 0.4118,  ..., 0.9333, 0.9216, 0.8784],
         [0.3804, 0.3529, 0.3804,  ..., 0.8706, 0.8510, 0.8039],
         [0.3922, 0.3451, 0.3647,  ..., 0.8157, 0.7843, 0.7451]],

        [[0.4824, 0.4980, 0.5255,  ..., 0.7059, 0.7098, 0.6824],
         [0.4745, 0.4941, 0.5176,  ..., 0.7098, 0.7255, 0.7216],
         [0.4588, 0.4510, 0.4510,  ..., 0.6902, 0.7059, 0.7059],
         ...,
         [0.2745, 0.2275, 0.2078,  ..., 0.7490, 0.7255, 0.6784],
         [0.2745, 0.2431, 0.2353,  ..., 0.6745, 0.6392, 0.5961],
         [0.2431, 0.2235, 0.2353,  ..., 0.5804, 0.5529, 0.5333]],

        [[0.7059, 0.6902, 0.6980,  ..., 0.8275, 0.8392, 0.8392],
         [0.7020, 0.7098, 0.7412,  ..., 0.8118, 0.8196, 0.8431],
         [0.7098, 0.7059, 0.7373,  ..., 0.8157, 0.8196, 0.

In [51]:
data_arr = ArrayDataset(img)

In [57]:
unet_model(img_transf(img).squeeze(0))

code/__torch__/torch/nn/modules/instancenorm.py:23: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
code/__torch__/torch/nn/modules/instancenorm/___torch_mangle_3.py:23: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
code/__torch__/torch/nn/modules/instancenorm/___torch_mangle_11.py:23: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
code/__torch__/torch/nn/modules/instancenorm/___torch_mangle_19.py:23: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
code/__torch__/torch/nn/modules/instancenorm/___torch_mangle_27.py:23: UserWarni

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/breast_cancer_segmentation/models/UNETModel.py", line 23, in forward
    input: Tensor) -> Tensor:
    net = self.net
    return (net).forward(input, )
            ~~~~~~~~~~~~ <--- HERE
  File "code/__torch__/monai/networks/nets/unet.py", line 21, in forward
    x: Tensor) -> Tensor:
    model = self.model
    return (model).forward(x, )
            ~~~~~~~~~~~~~~ <--- HERE
  File "code/__torch__/torch/nn/modules/container/___torch_mangle_64.py", line 15, in forward
    _2 = getattr(self, "2")
    input0 = (_0).forward(input, )
    input1 = (_1).forward(input0, )
              ~~~~~~~~~~~ <--- HERE
    return (_2).forward(input1, )
  def __len__(self: __torch__.torch.nn.modules.container.___torch_mangle_64.Sequential) -> int:
  File "code/__torch__/monai/networks/layers/simplelayers/___torch_mangle_54.py", line 13, in fo